In [ ]:
# Define the time variable
time_var = ncfile.variables['time']

# Convert the time variable to a human-readable format
time = nc.num2date(time_var[:], time_var.units)

# Specify the group
group_name = 'ZX_LIDAR_WLBZ_6'
group = ncfile.groups[group_name]

# Extract variables
wind_speed = group.variables['wind_speed'][:]
wind_from_direction = group.variables['wind_from_direction'][:]
wind_from_direction_availability = group.variables['wind_from_direction_availability'][:]
turbulence_intensity = group.variables['turbulence_intensity'][:]

# Create DataFrames for each height
height_indices = range(wind_speed.shape[3])  # height is the 4th dimension
dataframes = {}

for height_idx in height_indices:
    df = pd.DataFrame({
        'time': time,
        'wind_speed': wind_speed[:, 0, 0, height_idx],
        'wind_from_direction': wind_from_direction[:, 0, 0, height_idx],
        'wind_from_direction_availability': wind_from_direction_availability[:, 0, 0, height_idx],
        'turbulence_intensity': turbulence_intensity[:, 0, 0, height_idx]
    })
    
    # Set time as the index
    df.set_index('time', inplace=True)
    
    # Store the DataFrame in the dictionary
    dataframes[f'height_{height_idx}'] = df

# Heights in meters for each DataFrame
heights = [14, 42, 94, 140, 200, 250]

# Assign DataFrames to variables dynamically
for idx, height in enumerate(heights):
    globals()[f'height_{height}'] = dataframes[f'height_{idx}']

height_140

In [ ]:
# Height of interest 3. place in the 6. dimension of the ds = dataset
heights = [14, 42, 94, 140, 200, 250]
height_idx = 3

# Function to process buoy data for the specific height
def process_buoy_data(file_path, group_name):
    # Open the nc file
    ds = Dataset(file_path)
    
    # Extract the time variable
    time = ds.variables['time']
    time = nc.num2date(time[:], time.units)
    
    # Extract the group
    group = ds.groups[group_name]
    
    # Extract variables
    wind_speed = group.variables['wind_speed'][:]
    wind_from_direction = group.variables['wind_from_direction'][:]
    wind_from_direction_availability = group.variables['wind_from_direction_availability'][:]
    turbulence_intensity = group.variables['turbulence_intensity'][:]
    
    # Create DataFrame
    df = pd.DataFrame({
        'time': time,
        'wind_speed': wind_speed[:, 0, 0, height_idx],
        'wind_from_direction': wind_from_direction[:, 0, 0, height_idx],
        'wind_from_direction_availability': wind_from_direction_availability[:, 0, 0, height_idx],
        'turbulence_intensity': turbulence_intensity[:, 0, 0, height_idx]
    })
    
    # Set time as the index
    #df.set_index('time', inplace=True)
    
    return df

# Process buoy 6 data
bouy6_data = process_buoy_data(file_path, 'ZX_LIDAR_WLBZ_6')

# Process buoy 2 data
#bouy2_data = process_buoy_data(bouy2_path, 'ZX_LIDAR_WLBZ_2')

bouy6_data

In [ ]:
# Dictionary to store filtered DataFrames and statistics
filtered_dataframes = {}
nan_stats = {}

# Filter NaN values and calculate statistics
for idx, height in enumerate(heights):
    height_key = f'height_{height}'
    df = dataframes[f'height_{idx}']
    
    original_length = len(df)
    filtered_df = df.dropna()
    filtered_length = len(filtered_df)
    
    nan_count = original_length - filtered_length
    nan_percentage = (nan_count / original_length) * 100
    
    # Store the filtered DataFrame
    filtered_dataframes[height_key] = filtered_df
    
    # Store the statistics
    nan_stats[height_key] = {
        'filtered_nan_count': nan_count,
        'filtered_nan_percentage': nan_percentage
    }
# Assign filtered DataFrames to variables 
for height in heights:
    globals()[f'filter_height{height}'] = filtered_dataframes[f'height_{height}']

# Print NaN statistics for all heights
print("\nNaN Statistics for all heights:")
for height, stats in nan_stats.items():
    print(f"Height {height}: {stats['filtered_nan_count']} NaN values filtered ({stats['filtered_nan_percentage']:.2f}%)")

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Ensure the index is converted to datetime for all heights
for height in heights:
    df = globals()[f'filter_height{height}']
    df.index = pd.to_datetime([pd.Timestamp(t.isoformat()) for t in df.index])

# Define the date range
start_date = '2022-03-03'
end_date = '2022-03-17'

# Plot wind speed for all heights in one plot, within the specified date range
plt.figure(figsize=(14, 8))

for height in heights:
    df = globals()[f'filter_height{height}']
    df_time_selected = df.loc[start_date:end_date]
    plt.plot(df_time_selected.index, df_time_selected['wind_speed'], label=f'Height {height}m')

plt.xlabel('Time')
plt.ylabel('Wind Speed (m/s)')
plt.title('Wind Speed at Different Heights')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Define the date range
start_date = '2022-03-03'
end_date = '2023-03-03'

# Filter DataFrames for the specified date range
dfs_filtered = {}
for height in heights:
    df = globals()[f'filter_height{height}']
    dfs_filtered[height] = df.loc[start_date:end_date]

# Create bins with 0.5 m/s bin width
bin_width = 0.5
max_wind_speed = max([df['wind_speed'].max() for df in dfs_filtered.values()])
bins = np.arange(0, max_wind_speed + bin_width, bin_width)

height_pairs = [(14, 42), (94, 140), (200, 250)]

# Plot histograms for wind speed at the defined height pairs in subplots
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(14, 18), sharex=True)

for ax, (height1, height2) in zip(axes, height_pairs):
    df_filtered1 = dfs_filtered[height1]
    df_filtered2 = dfs_filtered[height2]
    
    ax.hist(df_filtered1['wind_speed'], bins=bins, alpha=0.5, label=f'Height {height1}m')
    ax.hist(df_filtered2['wind_speed'], bins=bins, alpha=0.5, label=f'Height {height2}m')
    
    ax.set_xlabel('Wind Speed (m/s)')
    ax.set_ylabel('Frequency')
    ax.set_title(f'Wind Speed Distribution at Heights {height1}m and {height2}m')
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
from windrose import WindroseAxes

# Function to plot wind rose
def plot_wind_rose(ax, df, height):
    ax.bar(df['wind_from_direction'], df['wind_speed'], normed=True, opening=0.8, edgecolor='white')
    ax.set_title(f'Wind Rose at Height {height}m')
    ax.set_legend()

# Create subplots for wind roses
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(18, 18), subplot_kw=dict(projection='windrose'))

# Plot wind roses for the defined heights
for ax, height in zip(axes.flatten(), heights):
    df = globals()[f'filter_height{height}']
    plot_wind_rose(ax, df, height)

plt.tight_layout()
plt.show()


In [ ]:
# Specify the main group
subgroup_name = ncfile.groups['METEO_WLBZ_6']

# Extract time variable from the main group
try:
    time_var = nc.num2date(subgroup_name.variables['time'][:], subgroup_name.variables['time'].units)
except KeyError:
    time_var = None

# Function to convert NetCDF subgroup to DataFrame
def nc_subgroup_to_df(subgroup, time_var=None):
    data_dict = {}
    if time_var is not None:
        data_dict['time'] = time_var
    
    for variable_name in subgroup.variables:
        variable = subgroup.variables[variable_name]
        data_dict[variable_name] = variable[:].flatten()  # Flatten to 1D array if necessary
    
    df = pd.DataFrame(data_dict)
    if 'time' in df.columns:
        df.set_index('time', inplace=True)
    return df

# Iterate over the subgroups in the main group and convert them to DataFrames
dfs = {}
for subgroup_name in subgroup_name.groups:
    subgroup = subgroup_name.groups[subgroup_name]
    df = nc_subgroup_to_df(subgroup, time_var)
    dfs[subgroup_name] = df
# Filter out NaN values from each DataFrame
filtered_dfs = {name: df.dropna() for name, df in dfs.items()}

vaisala_df = dfs['Vaisala Weather Station']
airmar_df = dfs['Airmar Weather Station']

airmar_df



In [ ]:
vaisala_df

## Wind farm field 

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Define the extent of the field and the surrounding area
field_extent = [5.10, 6.50, 54.10, 54.40]  # [min_lon, max_lon, min_lat, max_lat]
surrounding_extent = [2, 8, 51, 55]  # Larger area to encompass Netherlands, Germany, etc.


# Met mast positions
met_mast1_lon, met_mast1_lat = 5.521, 54.4 #Bouy6
met_mast2_lon, met_mast2_lat = 5.792, 54.5 #Bouy2

# Create a figure and axes
proj1=ccrs.LambertConformal(central_longitude=10.0, central_latitude=51, standard_parallels=(48, 54)) 
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(1, 1, 1, projection=proj1)
ax.set_extent(surrounding_extent, crs=ccrs.PlateCarree())

# Add map features
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.LAND, color='lightgray')

# Plot the square representing the farm field and the met masts 
min_lon, max_lon, min_lat, max_lat = field_extent
ax.plot([min_lon, max_lon, max_lon, min_lon, min_lon], 
        [min_lat, min_lat, max_lat, max_lat, min_lat], 
        color='red', linewidth=2, transform=ccrs.PlateCarree(), label='Wind fields N9.1-3')

ax.plot(met_mast1_lon, met_mast1_lat, 'bo', markersize=10, transform=ccrs.PlateCarree(), label='Bouy6')
ax.plot(met_mast2_lon, met_mast2_lat, 'go', markersize=10, transform=ccrs.PlateCarree(), label='Bouy2')
ax.legend()
ax.set_title('Field Area with Met Mast Positions')
plt.show()


In [ ]:
 fig, axes = plt.subplots(1, 2, figsize=(13, 8), sharey=True, sharex=True)

axes[0].plot(time6_filtered[4], ws6_150m)
axes[0].set_title('Wind Speed for interpolated 150 m at Buoy 6')
axes[0].set_xlabel('Time')
axes[0].set_ylabel('Wind Speed (m/s)')

axes[1].plot(time2_filtered[4], ws2_150m)
axes[1].set_title('Wind Speed for interpolated 150 m at Buoy 2')
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Wind Speed (m/s)')


# Adjust layout to prevent overlap
plt.tight_layout()

# Display the plots
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 8), sharey=True, sharex=True)

axes[0].plot(time6_filtered[4], wd6_150m)
axes[0].set_title('Wind Direction for interpolated 150 m at Buoy 6')
axes[0].set_xlabel('Time')
axes[0].set_ylabel('Direction in Degrees')

axes[1].plot(time2_filtered[4], wd2_150m)
axes[1].set_title('Wind Direction for interpolated 150 m at Buoy 2')
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Direction in Degrees')


# Adjust layout to prevent overlap
plt.tight_layout()

# Display the plots
plt.show()

In [ ]:
# Plotting time6 against ws_150m 
plt.figure(figsize=(10, 6))
plt.plot(time6_filtered[4], ws6_150m, label='Interpolated Data at 150m')
plt.xlabel('Time')
plt.ylabel('Interpolated Values')
plt.title('Time vs Interpolated Data at 150m')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
def interpolate_dataframe(df, col1, col2, height1, height2, target_height):
    # Calculate the interpolation factor
    factor = (target_height - height1) / (height2 - height1)
    # Perform the interpolation
    df[f'{col1}_{target_height}m'] = df[col1] + factor * (df[col2] - df[col1])
    return df

# Interpolating and adding the results to the dataframe for buoy6
filtered_buoy6 = interpolate_dataframe(filtered_buoy6, 'wind_speed_140m', 'wind_speed_200m', 140, 200, 150)
filtered_buoy6 = interpolate_dataframe(filtered_buoy6, 'wind_direction_140m', 'wind_direction_200m', 140, 200, 150)

# Interpolating and adding the results to the dataframe for buoy2
filtered_buoy2 = interpolate_dataframe(filtered_buoy2, 'wind_speed_140m', 'wind_speed_200m', 140, 200, 150)
filtered_buoy2 = interpolate_dataframe(filtered_buoy2, 'wind_direction_140m', 'wind_direction_200m', 140, 200, 150)

# Check the shapes
print(filtered_buoy6[f'wind_speed_140m_150m'].shape)
print(filtered_buoy6[f'wind_direction_140m_150m'].shape)
print()
print(filtered_buoy2[f'wind_speed_140m_150m'].shape)
print(filtered_buoy2[f'wind_direction_140m_150m'].shape)


In [ ]:
from scipy.interpolate import interp1d
from scipy.stats import weibull_min
from scipy.integrate import quad

# Constants
T = 8760  # total hours/year [h]
rho = 1.225  # air density [kg/m^3]
D = 240  # rotor diameter [m]
A = np.pi * (D / 2)**2  # swept area [m^2]

# Load data
wind_speed_data = df_interpol_height['ws6_150m']
power_output = pd.read_csv(turbine_power_curve_path)



def cut_in_windspeed(power_output):
    cut_in = power_output.loc[power_output['P'] > 0, 'ws'].min()
    return cut_in

def cut_out_windspeed(power_output):
    cut_out =  power_output.loc[power_output['P'] > 0, 'ws'].max()
    return cut_out

def interpolate_power_curve(wind_speeds, power_output):
    # Interpolate the power curve
    power_curve_interp = interp1d(wind_speeds, power_output, kind='cubic', fill_value='extrapolate')
    return power_curve_interp

def calculate_power_coefficient(power_curve_interp, v_range, rho, A):
    P_interp = power_curve_interp(v_range)
    c_p = P_interp / (0.5 * rho * A * wind_speed_data**3)
    return c_p

def fit_weibull(wind_speed_data):
    # Fit the Weibull distribution to the wind speed data
    shape, loc, scale = weibull_min.fit(wind_speed_data, floc=0)
    return shape, scale

def weibull_pdf(v, shape, scale):
    return weibull_min.pdf(v, shape, loc=0, scale=scale)


# Function to calculate the integrand for energy production
def integrand(v, power_curve_interp, rho, A, shape, scale):
    c_p = calculate_power_coefficient(power_curve_interp, wind_speed_data, rho, A)[0]
    weibull_prob = weibull_pdf(v, shape, scale)
    return 0.5 * rho * A * c_p * weibull_prob * v**3

# Function to calculate annual energy production
def calculate_annual_energy_production(power_curve_interp, rho, A, shape, scale, cut_in, cut_out):
    energy_production, _ = quad(integrand, cut_in, cut_out, args=(power_curve_interp, rho, A, shape, scale))
    return energy_production



#cut_in cut_ot 
cut_in = cut_in_windspeed(wind_speed_data)
cut_out = cut_out_windspeed(wind_speed_data)

#interpolate for c_p
power_curve_interp = interpolate_power_curve(power_output['ws'], power_output['P'])

# Fit the Weibull distribution to the wind speed data
shape, scale = fit_weibull(wind_speed_data)

# Calculate annual energy production
annual_energy_production = calculate_annual_energy_production(power_curve_interp, rho, A, shape, scale, cut_in, cut_out)

print(f"Annual Energy Production: {annual_energy_production} Wh")


In [ ]:
# Constants
T = 8760  # total hours/year [h]
rho = 1.225  # air density [kg/m^3]
D = 240  # rotor diameter [m]
A = np.pi * (D / 2)**2  # swept area [m^2]

# Load data
windspeed_data = df_interpol_height['ws6_150m']
power_output = pd.read_csv(turbine_power_curve_path)

def cut_in_windspeed(power_curve_data):
    return power_curve_data.loc[power_curve_data['P'] > 0, 'ws'].min()

def cut_out_windspeed(power_curve_data):
    return power_curve_data.loc[power_curve_data['P'] > 0, 'ws'].max()

def power_curve_interpolated(power_curve_data):
    return interpolate.interp1d(power_curve_data['ws'], power_curve_data['P'], fill_value="extrapolate")

def calculate_weibull_fit(windspeed_data):
    shape, _, scale = weibull_min.fit(windspeed_data, floc=0)  # floc=0 => location parameter defaults to 0
    return shape, scale

def weibull_pdf(ws, shape, scale):
    return weibull_min.pdf(ws, shape, loc=0, scale=scale)

def integrand(ws, shape, scale, power_curve_func):
    P = power_curve_func(ws)  # Use interpolated power curve values
    return P * weibull_pdf(ws, shape, scale)

def calculate_APP(shape, scale, power_curve_func, cut_in_ws, cut_out_ws):
    APP, error = integrate.quad(integrand, cut_in_ws, cut_out_ws, args=(shape, scale, power_curve_func), limit=100, epsabs=1e-05, epsrel=1e-05)
    return APP * T, error * T  # Multiply by total hours per year to get AEP

# Process data
cut_in_ws = cut_in_windspeed(power_output)
cut_out_ws = cut_out_windspeed(power_output)
#power_curve_func = power_curve_interpolated(power_curve_data)
shape, scale = calculate_weibull_fit(windspeed_data)

# Calculate APP (Annual Power Production)
APP, error = calculate_APP(shape, scale, power_curve_func, cut_in_ws, cut_out_ws)
print(f"APP of one Turbine: {APP / 1e6:.4f} GWh")  # Convert to MWh for readability
print(f"Estimated error: {error / 1e6:.4f} GWh")

turbines_area_of_interest_path = 'data/turbine-info/coordinates/area_of_interest/'
base_path = turbines_area_of_interest_path
file_N9_1 = f'{base_path}\layout-N-9.1.geom.csv'
file_N9_2 = f'{base_path}\layout-N-9.2.geom.csv'
file_N9_3 = f'{base_path}\layout-N-9.3.geom.csv'

# Load the data
data_N9_1 = pd.read_csv(file_N9_1)
data_N9_2 = pd.read_csv(file_N9_2)
data_N9_3 = pd.read_csv(file_N9_3)

print(f"Annual Energy Production of N-9.1 (133 Turbines): {((APP/1e9) * len(data_N9_1)):.4f} TWh")
print(f"Annual Energy Production of N-9.2 (133 Turbines): {((APP/1e9) * len(data_N9_2)):.4f} TWh")
print(f"Annual Energy Production of N-9.3 (100 Turbines): {((APP/1e9) * len(data_N9_3)):.4f} TWh")
print(f"Total Energy Production of all three fields (366 Turbines): {((APP/1e9) *366):.4f} TWh ")
print(f"This is {((((APP/1e9) *366)/507)*100):.2f} % of the electricity consumed in one yr in Germany.")

# Plotting the Weibull distribution and wind speed data
plt.figure(figsize=(10, 6))
ws_range = np.linspace(0, max(windspeed_data), 100)
weibull_pdf_values = weibull_pdf(ws_range, shape, scale)

plt.hist(windspeed_data, bins=30, density=True, alpha=0.6, color='blue', edgecolor='black', label='Wind Speed Data')
plt.plot(ws_range, weibull_pdf_values, 'r-', lw=2, label='Weibull Distribution')
plt.xlabel('Wind Speed (m/s)')
plt.ylabel('Density')
plt.title('Wind Speed Data and Weibull Distribution Fit')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Constants
T = 8760  # total hours/year [h]
rho = 1.225  # air density [kg/m^3]
D = 240  # rotor diameter [m]
A = np.pi * (D / 2)**2  # swept area [m^2]


def cut_in_windspeed(power_curve_data, power_curve_output):
    cut_in_ws = power_curve_data.loc[power_curve_output > 0, 'ws'].min()
    return cut_in_ws

def cut_out_windspeed(power_curve_data, power_curve_output):
    cut_out_ws = power_curve_data.loc[power_curve_output > 0, 'ws'].max()
    return cut_out_ws


def intergrand(measured_ws, power_curve_ws, power_curve_output):
    power_curve_interpolation = interp1d(power_curve_ws, power_curve_output, kind='linear', fill_value="extrapolate")
    interpolated_power = power_curve_interpolation(measured_ws)
    shape, loc, scale = stats.weibull_min.fit(measured_ws, floc=0)
    x = np.linspace(cut_in_ws, cut_out_ws, 100)
    pdf = stats.weibull_min.pdf(measured_ws, shape, loc, scale)
    return interpolated_power*pdf*A*rho*(measured_ws**3)

#Load data
power_curve_data = pd.read_csv(turbine_power_curve_path)
power_curve_ws = power_curve_data['ws']
power_curve_output = power_curve_data['P']
windspeed_data = df_interpol_height['ws6_150m'] #measured ws

# Calculate cut-in and cut-out wind speeds
cut_in_ws = cut_in_windspeed(power_curve_data, power_curve_output)
cut_out_ws = cut_out_windspeed(power_curve_data, power_curve_output)

# Perform the integration over the range from cut-in to cut-out wind speeds
annual_energy_production, _ = quad(intergrand(windspeed_data, power_curve_ws, power_curve_output), cut_in_ws, cut_out_ws, limit=100)

# Convert the energy production to MWh (since we integrated in terms of hours)
annual_energy_production_MWh = annual_energy_production / 1e6  # Convert from Wh to MWh

print(f"Annual energy production of one turbine: {annual_energy_production_MWh:.2f} MWh")


In [ ]:
import numpy as np

# Sample power curve data (wind speeds in m/s and power in kW)
wind_speeds = np.array([0, 3, 5, 8, 12, 15, 20, 25])
power_output = np.array([0, 0, 20, 100, 300, 400, 450, 450])

# Function to interpolate power output for a given wind speed
def interpolate_power(wind_speed, wind_speeds, power_output):
    if wind_speed <= wind_speeds[0]:
        return power_output[0]
    elif wind_speed >= wind_speeds[-1]:
        return power_output[-1]
    else:
        return np.interp(wind_speed, wind_speeds, power_output)

# Example wind speeds to interpolate
example_wind_speeds = [2, 6, 10, 18]

# Interpolating power outputs for example wind speeds
interpolated_powers = [interpolate_power(ws, wind_speeds, power_output) for ws in example_wind_speeds]

for ws, power in zip(example_wind_speeds, interpolated_powers):
    print(f"Wind Speed: {ws} m/s -> Power Output: {power} kW")


In [ ]:
import numpy as np
from scipy.interpolate import interp1d

# Sample power curve data (wind speeds in m/s and power in kW)
wind_speeds = np.array([0, 3, 5, 8, 12, 15, 20, 25])
power_output = np.array([0, 0, 20, 100, 300, 400, 450, 450])

# Create a linear interpolator
linear_interpolator = interp1d(wind_speeds, power_output, kind='linear', fill_value="extrapolate")

# Example wind speeds to interpolate
example_wind_speeds = [2, 6, 10, 18]

# Interpolating power outputs for example wind speeds using linear interpolation
interpolated_powers_linear = [linear_interpolator(ws) for ws in example_wind_speeds]

for ws, power in zip(example_wind_speeds, interpolated_powers_linear):
    print(f"Wind Speed: {ws} m/s -> Power Output (Linear): {power:.2f} kW")


In [ ]:
import numpy as np
from scipy.interpolate import interp1d

def calculate_power_coefficient(windspeed_data, power_windspeeds, power_output, rho, A):
    """
    Calculates the power coefficient for given wind speed data.

    Parameters:
    - windspeed_data: Array of wind speeds to interpolate.
    - power_windspeeds: Array of wind speeds from the power curve.
    - power_output: Array of power outputs corresponding to the wind speeds.
    - rho: Air density (kg/m^3).
    - A: Rotor swept area (m^2).

    Returns:
    - Array of power coefficients (Cp).
    """
    # Create a linear interpolator
    linear_interpolator = interp1d(power_windspeeds, power_output, kind='linear', fill_value="extrapolate")
    
    # Interpolate power outputs for the given wind speed data
    interpolated_powers = linear_interpolator(windspeed_data)
    
    # Calculate power coefficient (Cp)
    c_p = interpolated_powers / (0.5 * rho * A * windspeed_data**3)
    
    return c_p

# Example usage:
# Sample power curve data (wind speeds in m/s and power in kW)
power_windspeeds = np.array([0, 3, 5, 8, 12, 15, 20, 25])
power_output = np.array([0, 0, 20, 100, 300, 400, 450, 450])

# Example wind speed data to calculate Cp
windspeed_data = np.array([2, 6, 10, 18])

# Constants
rho = 1.225  # Air density in kg/m^3
A = 100  # Rotor swept area in m^2

# Calculating power coefficients for the example wind speed data
c_p_values = calculate_power_coefficient(windspeed_data, power_windspeeds, power_output, rho, A)

for ws, cp in zip(windspeed_data, c_p_values):
    print(f"Wind Speed: {ws} m/s -> Power Coefficient (Cp): {cp:.4f}")
